# 프롬프트 (Prompt)
`02_prompt.ipynb`

- LLM한테 주는 입력(지시, 맥락, 기억)
	- 지시: '~~해줘'
	- 맥락: Context - 현재 지시를 위해 제공하는 추가 정보
	- 기억: Memory - 지금까지 했던 대화 내용

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

In [5]:
# 추후에 체인. invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke 로 쓰게 됨)
prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [6]:
chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

'대한민국의 수도는 서울입니다.'

# Partial Variable (부분 변수)

- 프롬프트에서 매개변수 기본값 사용하기

In [7]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    imput_variables=['c1'],
    partial_variables={
        'c2': '미국'
				}
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다')

'한국과 미국의 수도는 각각 어디인가요?'

In [8]:
chain = prompt | llm
chain.invoke({'c1': 'kor', 'c2': 'us'}).content

'한국(대한민국)의 수도는 서울이고, 미국(United States)의 수도는 워싱턴 D.C.입니다.'

In [11]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [12]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

오늘은 9월 1일입니다. Sep 1 생일인 유명인으로 가장 잘 알려진 분은:
- Zendaya (1996-09-01)

다른 두 명도 함께 정확히 안내해 드리려면 추가 확인이 필요합니다. 인터넷 검색으로 확인해 세 명 모두 생년월일과 함께 확정해 드려도 될까요? 진행 여부를 알려 주세요.
===
다음은 오늘(1월 2일) 생일인 유명인 3명과 생년월일입니다.

- 아이작 아시모프 — 1920년 1월 2일
- 케이트 보스워스 — 1983년 1월 2일
- 쿠바 구딩 주니어 — 1968년 1월 2일


## `ChatPromptTemplate`
 - 채팅을 주고받는 템플릿 생성용
	- 대화 목록을 LLM에게 주입
	- 하나의 Chat 은 `role` 과 `message` 로 구성됨

In [16]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [17]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name}야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
				]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 gaida야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 gaida야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [18]:
# 한덩어리 테스트
llm.invoke(messages_cls).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'제 이름은 gaida예요. 반가워요! 무엇을 도와드릴까요?'

In [20]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '가이다', 'user_input': '배승은과 이기영의 이름점을 봐줘. 우리 이름의 조합으로 나온 궁합 해석까지 해줘'})

'좋아요! 재미로 보는 이름점과 궁합을 간단하게 계산해볼게요. 참고로 이것은 전통 점술의 한 방식 중 하나일 뿐, 과학적 예측은 아니고 즐길 거리로 봐주세요.\n\n1) 이름점(획수 추정) — 두 분의 총획수\n- 배승은 (배 + 승 + 은)\n  - 배: 6획\n  - 승: 5획\n  - 은: 4획\n  - 합계: 15획\n\n- 이기영 (이 + 기 + 영)\n  - 이: 2획\n  - 기: 3획\n  - 영: 4획\n  - 합계: 9획\n\n2) 궁합 해석(두 이름의 조합으로 본 간단한 해석)\n- 두 분의 총획 차이: |15 - 9| = 6\n- 궁합 점수(재미로 본 간단 공식): 100 - 2 × 차이 = 100 - 12 = 88점\n  - 해석 요지: 매우 좋은 궁합으로 보는 편에 가깝습니다. 서로의 강점이 잘 어울리며, 협력하면 큰 시너지를 낼 가능성이 큼.\n\n3) 간단한 해석 요약\n- 15획의 배승은: 추진력 강하고 열정적이며 리더십이 두드러짐. 새로운 도전을 즐김.\n- 9획의 이기영은: 차분하고 현실적이며 실무적 능력이 뛰어나고 인간관계 형성에 강함.\n- 두 사람의 조합: 서로 보완적이어서 함께 목표를 향해 잘 맞춰 나가면 좋은 결과를 낼 가능성이 큼. 다만 배승은이 추진을 강하게 밀고 나갈 때 이기영이 현실성이나 세부 실행에 관여해 균형을 맞추는 것이 중요할 수 있음.\n\n4) 더 다듬은 해석이 필요하신가요?\n- 원하시면 같은 이름점이라도 다른 방식으로 다시 계산해드릴 수 있어요. 예를 들면\n  - 초성/중성/종성 기준의 이름점\n  - 음양오행 기반의 해석\n  - 숫자 합계로 보는 간단한 궁합 등\n- 어떤 방식으로 보고 싶으신지 알려주시면 그 방식대로 다시 계산해 드리겠습니다.\n\n참고로 이 결과는 재미용으로만 받아들여 주세요. 더 깊은 해석이나 다른 방법으로도 원하신다면 말씀해 주세요!'

In [21]:
print('좋아요! 재미로 보는 이름점과 궁합을 간단하게 계산해볼게요. 참고로 이것은 전통 점술의 한 방식 중 하나일 뿐, 과학적 예측은 아니고 즐길 거리로 봐주세요.\n\n1) 이름점(획수 추정) — 두 분의 총획수\n- 배승은 (배 + 승 + 은)\n  - 배: 6획\n  - 승: 5획\n  - 은: 4획\n  - 합계: 15획\n\n- 이기영 (이 + 기 + 영)\n  - 이: 2획\n  - 기: 3획\n  - 영: 4획\n  - 합계: 9획\n\n2) 궁합 해석(두 이름의 조합으로 본 간단한 해석)\n- 두 분의 총획 차이: |15 - 9| = 6\n- 궁합 점수(재미로 본 간단 공식): 100 - 2 × 차이 = 100 - 12 = 88점\n  - 해석 요지: 매우 좋은 궁합으로 보는 편에 가깝습니다. 서로의 강점이 잘 어울리며, 협력하면 큰 시너지를 낼 가능성이 큼.\n\n3) 간단한 해석 요약\n- 15획의 배승은: 추진력 강하고 열정적이며 리더십이 두드러짐. 새로운 도전을 즐김.\n- 9획의 이기영은: 차분하고 현실적이며 실무적 능력이 뛰어나고 인간관계 형성에 강함.\n- 두 사람의 조합: 서로 보완적이어서 함께 목표를 향해 잘 맞춰 나가면 좋은 결과를 낼 가능성이 큼. 다만 배승은이 추진을 강하게 밀고 나갈 때 이기영이 현실성이나 세부 실행에 관여해 균형을 맞추는 것이 중요할 수 있음.\n\n4) 더 다듬은 해석이 필요하신가요?\n- 원하시면 같은 이름점이라도 다른 방식으로 다시 계산해드릴 수 있어요. 예를 들면\n  - 초성/중성/종성 기준의 이름점\n  - 음양오행 기반의 해석\n  - 숫자 합계로 보는 간단한 궁합 등\n- 어떤 방식으로 보고 싶으신지 알려주시면 그 방식대로 다시 계산해 드리겠습니다.\n\n참고로 이 결과는 재미용으로만 받아들여 주세요. 더 깊은 해석이나 다른 방법으로도 원하신다면 말씀해 주세요!')

좋아요! 재미로 보는 이름점과 궁합을 간단하게 계산해볼게요. 참고로 이것은 전통 점술의 한 방식 중 하나일 뿐, 과학적 예측은 아니고 즐길 거리로 봐주세요.

1) 이름점(획수 추정) — 두 분의 총획수
- 배승은 (배 + 승 + 은)
  - 배: 6획
  - 승: 5획
  - 은: 4획
  - 합계: 15획

- 이기영 (이 + 기 + 영)
  - 이: 2획
  - 기: 3획
  - 영: 4획
  - 합계: 9획

2) 궁합 해석(두 이름의 조합으로 본 간단한 해석)
- 두 분의 총획 차이: |15 - 9| = 6
- 궁합 점수(재미로 본 간단 공식): 100 - 2 × 차이 = 100 - 12 = 88점
  - 해석 요지: 매우 좋은 궁합으로 보는 편에 가깝습니다. 서로의 강점이 잘 어울리며, 협력하면 큰 시너지를 낼 가능성이 큼.

3) 간단한 해석 요약
- 15획의 배승은: 추진력 강하고 열정적이며 리더십이 두드러짐. 새로운 도전을 즐김.
- 9획의 이기영은: 차분하고 현실적이며 실무적 능력이 뛰어나고 인간관계 형성에 강함.
- 두 사람의 조합: 서로 보완적이어서 함께 목표를 향해 잘 맞춰 나가면 좋은 결과를 낼 가능성이 큼. 다만 배승은이 추진을 강하게 밀고 나갈 때 이기영이 현실성이나 세부 실행에 관여해 균형을 맞추는 것이 중요할 수 있음.

4) 더 다듬은 해석이 필요하신가요?
- 원하시면 같은 이름점이라도 다른 방식으로 다시 계산해드릴 수 있어요. 예를 들면
  - 초성/중성/종성 기준의 이름점
  - 음양오행 기반의 해석
  - 숫자 합계로 보는 간단한 궁합 등
- 어떤 방식으로 보고 싶으신지 알려주시면 그 방식대로 다시 계산해 드리겠습니다.

참고로 이 결과는 재미용으로만 받아들여 주세요. 더 깊은 해석이나 다른 방법으로도 원하신다면 말씀해 주세요!


## 프롬프팅
내가 원하는 답변을 구구절절 설명 X -> 예시를 주고 답변을 형태를 유도
- Zero-shot prompting : 예시 없이 질문만 -> 바로 답변
- One-shot prompting : 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting** : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화 해서 답변

In [26]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 1998년에 몇 살이었나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났으므로, 1998년에는 42세였습니다.  
최종 답변은: 42살입니다.


## `langchain-hub`
다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [27]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [25]:
prompt = PromptTemplate.from_template(
    '시작은 미약하지만 끝은 창대하리'
)

hub.push('gaida/prompt-hub-test', prompt)

LangSmithUserError: Cannot create a prompt for another tenant.
Current tenant: None,
Requested tenant: gaida